In [1]:
from IPython.display import clear_output

In [3]:
import warnings
warnings.filterwarnings("ignore")

import os 
import torch

import lightning.pytorch as pl
from config import get_config
from lt_datamodule import OpusBooksDataModule
from lt_dataset import BiLangLitModule

In [10]:
config = get_config()
config.update({
  "project_name" : "ByLanguage-EN_FN", # Wandb project name
  "run_name" : "trial", # wandb run name
  "batch_size" : 32,
  "lang_src" : "en",
  "lang_tgt" : "fr",
  "seq_len" : 160,
  "num_epochs" : 10,
  "lr": 1e-3
})

In [11]:
datamodule = OpusBooksDataModule(config, 8, True)

In [12]:
datamodule.prepare_data()
datamodule.setup("")

Found cached dataset opus_books (C:/Users/pbosamia/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)
Found cached dataset opus_books (C:/Users/pbosamia/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)


Tokenizer found. Loading from file...
Tokenizer found. Loading from file...
[x] Maximum Length of Sentence:-
	[*] source sentence: 471
	[*] target sentence: 482


In [16]:
model = BiLangLitModule(config["lr"], config, datamodule.tokenizer_src, datamodule.tokenizer_tgt)


In [17]:
parameter_sharing=True

In [19]:
from lightning.pytorch.callbacks import (
    LearningRateMonitor,
    RichProgressBar,
)
from lightning.pytorch.loggers import WandbLogger


callbacks = [LearningRateMonitor(logging_interval="step")]
logger = WandbLogger(project=config["project_name"], name=config["run_name"])
trainer = pl.Trainer(
    callbacks=callbacks,
    logger=logger,
    max_epochs=config["num_epochs"],
    accelerator = "cpu",
)
trainer.fit(model=model, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Found cached dataset opus_books (C:/Users/pbosamia/.cache/huggingface/datasets/opus_books/en-fr/1.0.0/e8f950a4f32dc39b7f9088908216cd2d7e21ac35f893d04d39eb594746af2daf)

  | Name            | Type             | Params
-----------------------------------------------------
0 | net             | Transformer      | 90.2 M
1 | criterion       | CrossEntropyLoss | 0     
2 | char_error_rate | _CharErrorRate   | 0     
3 | word_error_rate | _WordErrorRate   | 0     
4 | bleu_score      | _BLEUScore       | 0     
-----------------------------------------------------
90.2 M    Trainable params
0         Non-trainable params
90.2 M    Total params
360.724   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

ValueError: Caught ValueError in DataLoader worker process 2.
Original Traceback (most recent call last):
  File "c:\Users\pbosamia\Anaconda3\lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\pbosamia\Anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\pbosamia\Anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\pbosamia\OneDrive - Deloitte (O365D)\Documents\School_of_AI\S16_Speeding_Up_Transformers\dataset.py", line 44, in __getitem__
    raise ValueError("Sentence is too long")
ValueError: Sentence is too long
